In [49]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)   
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing

In [50]:
with open("data/Data_120294_2023-03-29.csv", "r") as f:
    d = pd.read_csv(f)

d.head()

/tmp/ipykernel_10184/2003393883.py:2: DtypeWarning: Columns (2,64,91,94,97,106,108,109,110,111,112,114,116,119,120,122,127,139,142,143,144,145,152,153,154,155,156,157,159,160,161,163,164,165,169,170,171,172,175,178,179,180,187,189,215,216,217,218,219,317,318) have mixed types. Specify dtype option on import or set low_memory=False.
  d = pd.read_csv(f)


,wl_org,cod_wl,cod_ostxt_wl,num_prev_tx,thoracic_dgn,tah,vas,onvent,icu,inotropic,gender,abo,wgt_kg_tcr,hgt_cm_tcr,bmi_tcr,education,ecmo_tcr,iabp_tcr,inhaled_no,inotropes_tcr,vad_device_ty_tcr,vad_brand1_tcr,vad_brand2_tcr,vad_tah_tcr,func_stat_tcr,tcr_dgn,diab,dial_ty_tcr,cereb_vasc,malig_tcr,malig_ty_tcr,most_rcnt_creat,tot_serum_album,sud_death,impl_defibril,hemo_sys_tcr,hemo_pa_dia_tcr,hemo_pa_mn_tcr,hemo_pcw_tcr,hemo_co_tcr,cig_use,tcr_dur_abstain,prior_card_surg_tcr,histry_cig_old,sternotomy_tcr,ssdmf_death_date,days_stat1,days_stat1a,days_stat2,days_stat1b,days_stata4,days_stata5,days_stata2,days_stata3,days_stata1,days_stata6,init_stat,init_creat,rem_cd,dayswait_chron,end_stat,init_age,life_sup_tcr,activate_date,death_date,end_date,init_date,ethnicity,ethcat,pt_code,init_hgt_cm_calc,init_wgt_kg_calc,init_bmi_calc,end_hgt_cm_calc,end_wgt_kg_calc,end_bmi_calc,composite_death_date,ventilator_tcr,lvad_at_listing,lvad_while_listed,rvad_at_listing,rvad_while_listed,work_income_tcr,academic_prg_tcr,academic_level_tcr,exercise_o2,academic_level_trr,academic_prg_trr,func_stat_trr,med_cond_trr,ecmo_trr,work_income_trr,cognitive_dev_trr,motor_dev_trr,measurement_date_trr,pge_trr,creat_trr,dial_after_list,fev1_trr,fvc_trr,hemo_co_trr,hemo_pa_dia_trr,hemo_pa_mn_trr,hemo_pcw_trr,hemo_sys_trr,iabp_trr,infect_iv_drug_trr,inotropes_trr,inotrop_vaso_co_trr,inotrop_vaso_dia_trr,inotrop_vaso_mn_trr,inotrop_vaso_pcw_trr,inotrop_vaso_sys_trr,pco2_trr,steroid,tbili,transfusions,vad_device_ty_trr,prior_card_surg_type_trr,prior_card_surg_type_ostxt_trr,cmv_status,cod,cod_ostxt,gstatus,gtime,lastfuno,ptime,px_stat,func_stat_trf,amis,bmis,drmis,hlamis,pramr,prapk,pramr_cl1,pramr_cl2,prapk_cl1,prapk_cl2,hist_cocaine_don,age_don,ethcat_don,abo_don,alcohol_heavy_don,gender_don,cod_ostxt_don,bun_don,creat_don,sgot_don,sgpt_don,tbili_don,hypertens_dur_don,cancer_oth_ostxt_don,contin_cig_don,contin_cocaine_don,contin_oth_drug_don,diet_don,diuretics_don,diabdur_don,hist_hypertens_don,hist_cancer_don,hist_insulin_dep_don,hist_diabetes_don,diabetes_don,hist_oth_drug_don,insulin_don,hgt_cm_don_calc,wgt_kg_don_calc,bmi_don_calc,px_stat_date,tx_date,discharge_date,tx_type,abo_mat,age,dial_prior_tx,ischtime,grf_fail_cause,grf_fail_date,grf_stat,age_group,hgt_cm_calc,wgt_kg_calc,bmi_calc,distance,vad_tah_trr,tx_year,referral_date,pco2_don,pulm_cath_don,map_init_don,map_post_don,cvp_cath_init_don,cvp_cath_post_don,cvp_cath_old_don,pcwp_init_don,pcwp_post_don,svr_init_don,svr_post_don,syst_pa_cath_init_don,syst_pa_cath_post_don,systolic_pa_cath_old_don,diast_pa_cath_init_don,diast_pa_cath_post_don,diastolic_pa_cath_old_don,cardiac_output_cath_init_don,cardiac_output_cath_post_don,cardiac_output_cath_old_don,card_idx_init_don,card_idx_post_don,broncho_lt_don,broncho_rt_don,chest_xray_don,ph_don,hematocrit_don,abn_valves_don,abn_lvh_don,abn_congen_don,wall_abn_seg_don,wall_abn_glob_don,data_transplant,data_waitlist,alg_ind,alg_maint,cyclosporin_ind,cyclosporin_maint,okt4_ind,okt4_maint,cytoxan_ind,cytoxan_maint,methotrexate_ind,methotrexate_maint,brequinarsodium_ind,brequinarsodium_maint,mizoribine_ind,mizoribine_maint,atgam_ind,atgam_maint,nratgnrats_ind,nratgnrats_maint,okt3_ind,okt3_maint,xomazymecd5_ind,xomazymecd5_maint,dab486il2_ind,dab486il2_maint,antilfa1_ind,antilfa1_maint,antiicam1_ind,antiicam1_maint,il1receptorantagonist_ind,il1receptorantagonist_maint,antiil6_ind,antiil6_maint,antitnf_ind,antitnf_maint,solubleil1receptor_ind,solubleil1receptor_maint,aldesleukin_ind,aldesleukin_maint,t10b9_ind,t10b9_maint,sandimmune_ind,sandimmune_maint,deoxyspergualin_ind,deoxyspergualin_maint,neoral_ind,neoral_maint,sangcya_ind,sangcya_maint,thymoglobulin_ind,thymoglobulin_maint,zenapax_ind,zenapax_maint,simulect_ind,simulect_maint,gengraf_ind,gengraf_maint,zortress_ind,zortress_maint,eon_ind,eon_maint,myfortic_ind,myfortic_maint,genericcyclosporin_ind,genericcyclosporin_maint,steroids_ind,steroids_maint,prograf_ind,prograf_maint,campath_ind,campath_maint,fty720_in

In [51]:
print(d.education.dtype)
print(d.education.unique())

float64
[ nan   6.   5.   4.   3.   2. 996. 998.   1.]


There seems to exists columns that appear numeric but are actually categorical. Let's attempt to identify columns that are strictly numerical.

In the provided documentation XLS file, numerical columns do not have a "SAS ANALYSIS FORMAT". The "SAS FORMAT" columns also gives information about how the data is formatted. Let's use this information to categorize the columns.

In [52]:
# fetch data from excel file
with open("data/Documentation.xls", "rb") as f:
    docs = pd.read_excel(f, sheet_name=None, header=None)
docs[list(docs.keys())[0]].head()

,0,1,2
0,Worksheet Tab Name,Description of Contents,Relative File (s)
1,File Formats,Explanation of the data file formats within folders provided on the disc,ALL
2,MAIN DOCUMENTATION,Descriptions for individual files with hyperlinks to data field documentation,ALL
3,Formats (Lookup Values),Provides locations for SAS formats and lookup values for variables,ALL
4,Additional HLA,Data dictionary for HLA data (all organ types),All Organs


In [53]:
def is_data_sheet(docs, sheet):
    if "SAS ANALYSIS FORMAT" in docs[sheet].values[1] \
    and "FORMAT".lower() not in sheet.lower():
        return True
    return False

def find_column_index(docs, sheet, column_name):
    return list(docs[sheet].values[1]).index(column_name)

numerical_columns = set()
categorical_columns = set()
date_columns = set()
datetime_columns = set()
string_columns = set()
unknown_type_columns = set()

for sheet in docs:
    if is_data_sheet(docs, sheet):
        sas_index = find_column_index(docs, sheet, "SAS ANALYSIS FORMAT")
        format_index = find_column_index(docs, sheet, "SAS FORMAT")
        type_index = find_column_index(docs, sheet, "DATA TYPE")

        for row in docs[sheet].values[2:]:
            if str(row[type_index]).upper() == "NUM":
                if "MMDDYY" in str(row[format_index]) \
                or "MMDDYY" in str(row[sas_index]):
                    date_columns.add(row[0].lower())
                elif "DATETIME" in str(row[format_index]) \
                or "DATETIME" in str(row[sas_index]):
                    datetime_columns.add(row[0].lower())
                elif isinstance(row[sas_index], float) and np.isnan(row[sas_index]):
                    numerical_columns.add(row[0].lower())
                else:
                    categorical_columns.add(row[0].lower())
            elif str(row[type_index]).upper() == "CHAR":
                if row[sas_index]:
                    categorical_columns.add(row[0].lower())
                else:
                    string_columns.add(row[0].lower())
            elif row[type_index] == "DATE":
                date_columns.add(row[0].lower())
            else:
                unknown_type_columns.add(row[0].lower())


print("Numerical columns:", numerical_columns)
print("Categorical columns:", categorical_columns)
print("Date columns", date_columns)
print("Datetime columns:", datetime_columns)
print("String columns:", string_columns)
print("Unknown type columns:", unknown_type_columns)

Numerical columns: {'init_meld_peld_lab_score', 'bmis', 'criteriadurabledevsupport', 'amylase_don', 'currtherdopamine', 'preop_pao2', 'distance', 'num_in_disc', 'hemo_pcw_tcr', 'frac_extrem_num', 'iabpwithouthemoartlac', 'inocardiacindex', 'mr_serum_creat', 'dayswait_chron_ki', 'init_cpra', 'on_vent_trr', 'sendatapramethodst', 'cardiomyopathynyhapcwp', 'preop_bili', 'extinomuldobutamine', 'list_year', 'age_diab', 'fev_post_pct', 'pulm_inf_don', 'init_bmi_calc', 'inhaled_no', 'init_albumin', 'max_expanded_mismatch_ab', 'doncrit_max_warm_time', 'growth4', 'end_blu_flg', 'artificial_li', 'reop_oth_li', 'syst_pa_cath_init_don', 'sarcoma', 'hrsevfailalbuminst', 'frac_extrem', 'mcsdwithhemo', 'inomulepinephrine', 'low_pco2_ph', 'hrsevfailbilirubin', 'iabpcapwedgepressure', 'pramr_cl2', 'wgt_kg', 'pco2_trr', 'preop_urine_ratio', 'mcsdwithrhfprostacyclin', 'other_dev_therapy', 'high_pco2_po2', 'drug50_days', 'sgpt_don', 'wl_id_code', 'cold_isch_tm', 'iabpwithouthemo', 'orig_tumor_size3', 'brai

Let's investigate the columns with unknown type.

In [54]:
for column in unknown_type_columns:
    if column in d.columns:
        print(column, d[column].dtype, d[column].unique())

None of these columns are in the dataset, so we are going to ignore them.

Let's explore how well the documentation describes the data set.

In [55]:

col_union = set.union(numerical_columns, categorical_columns, date_columns, datetime_columns, string_columns, unknown_type_columns)
print("Number columns in the documentation:", len(col_union))

print("Number columns in dataset:", len(d.columns))

intersect = col_union & set(d.columns)
print("Number shared columns:", len(intersect))

# check which columns are not described in the documentation
non_described_columns = set(d.columns) - intersect
print("Number undocumented:", len(non_described_columns))
print("Undocumented columns:", non_described_columns)


Number columns in the documentation: 2843
Number columns in dataset: 320
Number shared columns: 222
Number undocumented: 98
Undocumented columns: {'fty720_ind', 'okt4_maint', 'leflunomide_maint', 'atgam_maint', 'cyclosporin_ind', 'mizoribine_maint', 'genericcyclosporin_maint', 'okt4_ind', 'antilfa1_ind', 'fty720_maint', 't10b9_ind', 'steroids_maint', 'sangcya_maint', 'okt3_maint', 'dab486il2_ind', 'okt3_ind', 'rituxan_ind', 'thymoglobulin_maint', 'genericmyfortic_maint', 'astagrafxl_maint', 'nratgnrats_ind', 'alg_maint', 'zortress_ind', 'deoxyspergualin_maint', 'eon_maint', 'steroids_ind', 'cellcept_ind', 'gengraf_ind', 'myfortic_maint', 'atgam_ind', 't10b9_maint', 'cytoxan_maint', 'envarsusxr_ind', 'genericsirolimus_ind', 'sangcya_ind', 'campath_maint', 'rapamune_ind', 'prograf_ind', 'solubleil1receptor_ind', 'antiil6_ind', 'methotrexate_ind', 'oth_drug_ind1', 'antiicam1_maint', 'cyclosporin_maint', 'dab486il2_maint', 'neoral_ind', 'eon_ind', 'oth_drug_ind2', 'gengraf_maint', 'sandimm

There are apparently many columns that are not described in the documentation, or are somehow renamed. Before categorizing these, let's remove statistically insignificant columns from the dataset entirely. 

In [56]:
# list all columns with more than 95% missing values
many_missing = d.isnull().sum()[d.isnull().sum() > 0.95 * len(d)]
print(many_missing.index)
print("\n%d out of %d total columns are missing values in >95%% of rows" 
    % (len(many_missing), len(d.columns)))

Index(['cod_wl', 'cod_ostxt_wl', 'vad_brand2_tcr', 'malig_ty_tcr',
       'init_creat', 'lvad_while_listed', 'rvad_at_listing',
       'rvad_while_listed', 'exercise_o2', 'cognitive_dev_trr',
       ...
       'rapamune_ind', 'rapamune_maint', 'leflunomide_ind',
       'leflunomide_maint', 'oth_drug_ind1', 'oth_drug_maint1',
       'oth_drug_ind2', 'oth_drug_maint2', 'oth_drug_ostxt1',
       'oth_drug_ostxt2'],
      dtype='object', length=124)

124 out of 320 total columns are missing values in >95% of rows


In [57]:
# remove columns with more than 95% missing values
d = d.drop(many_missing.index, axis=1)

Let's also remove columns with only one unique value, as they are not significant.

In [58]:
# list columns with only one unique value
one_unique = d.nunique()[d.nunique(dropna=True) == 1]
print(one_unique.index)

Index(['wl_org', 'lvad_at_listing', 'tx_type', 'data_waitlist',
       'generictacrolimus_maint', 'genericmmf_maint'],
      dtype='object')


In [59]:
d = d.drop(one_unique.index, axis=1)

In [60]:
# print columns and their percentage of missing values, sorted
missing = d.isnull().sum()[d.isnull().sum() > 0]
print(missing.sort_values(ascending=False) / len(d))

prior_card_surg_type_ostxt_trr   0.94990
vad_tah_trr                      0.94982
grf_fail_date                    0.94688
grf_fail_cause                   0.94643
academic_level_trr               0.94564
academic_prg_trr                 0.94564
cod_ostxt_don                    0.94488
academic_prg_tcr                 0.94214
academic_level_tcr               0.94214
diuretics_don                    0.93762
diet_don                         0.93762
hypertens_dur_don                0.93728
cod_ostxt                        0.93727
contin_cocaine_don               0.93432
death_date                       0.92809
prior_card_surg_type_trr         0.92654
simulect_ind                     0.91551
simulect_maint                   0.91551
cyclosporin_maint                0.91461
cyclosporin_ind                  0.91461
thymoglobulin_ind                0.91218
thymoglobulin_maint              0.91218
vad_brand1_tcr                   0.88447
neoral_maint                     0.87989
neoral_ind      

In [61]:
print(len(d.columns), "columns left")

190 columns left


Let's revisit the the columns not described in the dataset.

In [62]:
# check which columns are not described in the documentation
non_described_columns = set(d.columns) - intersect
print("Number columns missing from documentation:", len(non_described_columns))
print("Columns not described in documentation:", non_described_columns)

Number columns missing from documentation: 17
Columns not described in documentation: {'imuran_maint', 'steroids_ind', 'simulect_maint', 'neoral_ind', 'cellcept_ind', 'thymoglobulin_maint', 'prograf_maint', 'cyclosporin_maint', 'thymoglobulin_ind', 'cellcept_maint', 'steroids_maint', 'neoral_maint', 'cyclosporin_ind', 'prograf_ind', 'imuran_ind', 'simulect_ind', '_merge'}


Let's investigate the types of these columns.

In [63]:
for col in non_described_columns:
    # print unique values and counts of each column
    print(d[col].value_counts())

1.00000    17934
0.00000     1663
Name: imuran_maint, dtype: int64
1.00000    49656
0.00000    19133
Name: steroids_ind, dtype: int64
0.00000    10107
1.00000       54
Name: simulect_maint, dtype: int64
0.00000    11197
1.00000     3248
Name: neoral_ind, dtype: int64
0.00000    33356
1.00000     2755
Name: cellcept_ind, dtype: int64
0.00000    10516
1.00000       46
Name: thymoglobulin_maint, dtype: int64
1.00000    26215
0.00000      110
Name: prograf_maint, dtype: int64
1.00000    10117
0.00000      152
Name: cyclosporin_maint, dtype: int64
1.00000    10004
0.00000      558
Name: thymoglobulin_ind, dtype: int64
1.00000    35817
0.00000      294
Name: cellcept_maint, dtype: int64
1.00000    64044
0.00000     4745
Name: steroids_maint, dtype: int64
1.00000    14116
0.00000      329
Name: neoral_maint, dtype: int64
1.00000    9467
0.00000     802
Name: cyclosporin_ind, dtype: int64
0.00000    25773
1.00000      552
Name: prograf_ind, dtype: int64
1.00000    13158
0.00000     6439
Name: 

From this data, it is clear that all of these columns are categorical. Let's group them with the other columns.

In [64]:
for col in non_described_columns:
    categorical_columns.add(col)
unknown_type_columns = []

From experimentation, the columns 'grf_fail_date' and 'tah' were both identified as being misdocumented as numerical columns.
The columns 'cod_ostxt', 'cod_ostxt_don' and 'prior_card_surg_type_ostxt_trr' were identified as beng text-type fields. These, we can ignore.

Let's put the columns in their correct categories.

In [65]:
date_columns.add("grf_fail_date")
numerical_columns.remove("grf_fail_date")

categorical_columns.add("tah")
numerical_columns.remove("tah")

categorical_columns.remove("cod_ostxt")
categorical_columns.remove("cod_ostxt_don")
categorical_columns.remove("prior_card_surg_type_ostxt_trr")

Let's focus on only columns that exist in the dataset from now on.

In [66]:
d_numerical_columns = list(numerical_columns & set(d.columns))
d_categorical_columns = list(categorical_columns & set(d.columns))
d_date_columns = list(date_columns & set(d.columns))
d_datetime_columns = list(datetime_columns & set(d.columns))

print("Number numerical columns:", len(d_numerical_columns))
print("Number categorical columns:", len(d_categorical_columns))
print("Number date columns:", len(d_date_columns))
print("Number datetime columns:", len(d_datetime_columns))

Number numerical columns: 82
Number categorical columns: 97
Number date columns: 11
Number datetime columns: 0


There are apparently no datetime columns in the dataset; only in the documentation. Let's ignore these from now on.

Let's impute missing values.

In [67]:
from sklearn.impute import SimpleImputer

# impute missing numerical values with the mean
numerical_imputer = SimpleImputer(strategy="mean")
for col in d_numerical_columns:
    try:
        d[col] = numerical_imputer.fit_transform(d[col].values.reshape(-1, 1))
    except:
        print("Error imputing column", col)

# impute missing categorical values with the most frequent value
categorical_imputer = SimpleImputer(strategy="most_frequent")
d[d_categorical_columns] = categorical_imputer.fit_transform(d[d_categorical_columns])

# TODO: impute date values with the mean somehow

# impute missing date values with the most frequent value
date_imputer = SimpleImputer(strategy="most_frequent")
d[d_date_columns] = date_imputer.fit_transform(d[d_date_columns])

Now, let's normalize numerical values.

In [68]:
# Normalize numerical columns using sklearn
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
d[d_numerical_columns] = scaler.fit_transform(d[d_numerical_columns])

Let's also encode categorical columns using One-Hot encoding.

In [69]:
# one hot encode categorical columns
d = pd.get_dummies(d, columns=d_categorical_columns)

/tmp/ipykernel_10184/953359956.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  d = pd.get_dummies(d, columns=d_categorical_columns)
/tmp/ipykernel_10184/953359956.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  d = pd.get_dummies(d, columns=d_categorical_columns)
/tmp/ipykernel_10184/953359956.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  d = pd.get_dummies(d, columns=d_categorical_columns)
/tmp/ipykernel_10184/953359956.py:2: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  d = pd.get_dummies(d, columns=d_categorical_columns)
/tmp/ipykernel_10184/953359956.p

In [70]:
d.head()

,num_prev_tx,wgt_kg_tcr,hgt_cm_tcr,bmi_tcr,ecmo_tcr,iabp_tcr,inhaled_no,inotropes_tcr,vad_brand1_tcr,most_rcnt_creat,tot_serum_album,hemo_sys_tcr,hemo_pa_dia_tcr,hemo_pa_mn_tcr,hemo_pcw_tcr,hemo_co_tcr,sternotomy_tcr,ssdmf_death_date,days_stat1,days_stat1a,days_stat2,days_stat1b,days_stata4,days_stata5,days_stata2,days_stata3,days_stata1,days_stata6,dayswait_chron,init_age,activate_date,death_date,end_date,init_date,pt_code,init_hgt_cm_calc,init_wgt_kg_calc,init_bmi_calc,end_hgt_cm_calc,end_wgt_kg_calc,end_bmi_calc,composite_death_date,ventilator_tcr,ecmo_trr,pge_trr,creat_trr,hemo_co_trr,hemo_pa_dia_trr,hemo_pa_mn_trr,hemo_pcw_trr,hemo_sys_trr,iabp_trr,inotropes_trr,tbili,prior_card_surg_type_ostxt_trr,cod_ostxt,gstatus,gtime,lastfuno,ptime,amis,bmis,drmis,hlamis,pramr,prapk,pramr_cl1,pramr_cl2,age_don,cod_ostxt_don,bun_don,creat_don,sgot_don,sgpt_don,tbili_don,hgt_cm_don_calc,wgt_kg_don_calc,bmi_don_calc,px_stat_date,tx_date,discharge_date,age,ischtime,grf_fail_date,hgt_cm_calc,wgt_kg_calc,bmi_calc,distance,tx_year,referral_date,pco2_don,ph_don,hematocrit_don,func_stat_trf_1.0,func_stat_trf_2.0,func_stat_trf_3.0,func_stat_trf_996.0,func_stat_trf_998.0,func_stat_trf_2010.0,func_stat_trf_2020.0,func_stat_trf_2030.0,func_stat_trf_2040.0,func_stat_trf_2050.0,func_stat_trf_2060.0,func_stat_trf_2070.0,func_stat_trf_2080.0,func_stat_trf_2090.0,func_stat_trf_2100.0,func_stat_trf_4010.0,func_stat_trf_4020.0,func_stat_trf_4030.0,func_stat_trf_4040.0,func_stat_trf_4050.0,func_stat_trf_4060.0,func_stat_trf_4070.0,func_stat_trf_4080.0,func_stat_trf_4090.0,func_stat_trf_4100.0,work_income_tcr_N,work_income_tcr_U,work_income_tcr_Y,inotrop_vaso_co_trr_N,inotrop_vaso_co_trr_Y,hist_cancer_don_N,hist_cancer_don_U,hist_cancer_don_Y,cyclosporin_ind_0.0,cyclosporin_ind_1.0,data_transplant_N,data_transplant_Y,dial_prior_tx_N,dial_prior_tx_U,dial_prior_tx_Y,tah_N,tah_Y,pulm_cath_don_N,pulm_cath_don_Y,infect_iv_drug_trr_N,infect_iv_drug_trr_U,infect_iv_drug_trr_Y,inotrop_vaso_pcw_trr_N,inotrop_vaso_pcw_trr_Y,grf_stat_N,grf_stat_Y,steroids_maint_0.0,steroids_maint_1.0,cig_use_N,cig_use_Y,thymoglobulin_maint_0.0,thymoglobulin_maint_1.0,inotropic_N,inotropic_Y,academic_level_trr_1.0,academic_level_trr_2.0,academic_level_trr_3.0,academic_level_trr_4.0,academic_level_trr_996.0,academic_level_trr_998.0,broncho_lt_don_1.0,broncho_lt_don_2.0,broncho_lt_don_3.0,broncho_lt_don_4.0,broncho_lt_don_5.0,broncho_lt_don_6.0,broncho_lt_don_7.0,broncho_lt_don_998.0,life_sup_tcr_N,life_sup_tcr_Y,alcohol_heavy_don_N,alcohol_heavy_don_U,alcohol_heavy_don_Y,ethnicity_0.0,ethnicity_1.0,steroids_ind_0.0,steroids_ind_1.0,cellcept_ind_0.0,cellcept_ind_1.0,func_stat_trr_1.0,func_stat_trr_2.0,func_stat_trr_3.0,func_stat_trr_996.0,func_stat_trr_998.0,func_stat_trr_2010.0,func_stat_trr_2020.0,func_stat_trr_2030.0,func_stat_trr_2040.0,func_stat_trr_2050.0,func_stat_trr_2060.0,func_stat_trr_2070.0,func_stat_trr_2080.0,func_stat_trr_2090.0,func_stat_trr_2100.0,func_stat_trr_4010.0,func_stat_trr_4020.0,func_stat_trr_4030.0,func_stat_trr_4040.0,func_stat_trr_4050.0,func_stat_trr_4060.0,func_stat_trr_4070.0,func_stat_trr_4080.0,func_stat_trr_4090.0,func_stat_trr_4100.0,broncho_rt_don_1.0,broncho_rt_don_2.0,broncho_rt_don_3.0,broncho_rt_don_4.0,broncho_rt_don_5.0,broncho_rt_don_6.0,broncho_rt_don_7.0,broncho_rt_don_998.0,thoracic_dgn_2,thoracic_dgn_14,thoracic_dgn_100,thoracic_dgn_999,thoracic_dgn_1000,thoracic_dgn_1001,thoracic_dgn_1002,thoracic_dgn_1003,thoracic_dgn_1004,thoracic_dgn_1005,thoracic_dgn_1006,thoracic_dgn_1007,thoracic_dgn_1049,thoracic_dgn_1050,thoracic_dgn_1051,thoracic_dgn_1052,thoracic_dgn_1053,thoracic_dgn_1054,thoracic_dgn_1099,thoracic_dgn_1100,thoracic_dgn_1101,thoracic_dgn_1102,thoracic_dgn_1103,thoracic_dgn_1104,thoracic_dgn_1105,thoracic_dgn_1106,thoracic_dgn_1199,thoracic_dgn_1200,thoracic_dgn_1201,thoracic_dgn_1202,thoracic_dgn_1203,thoracic_dgn_1204,thoracic_dgn_1205,thoracic_dgn_1206,thoracic_dgn_1207,thoracic_dgn_1208,thoracic_dgn_1209,thoracic_dgn_1497,

In [71]:
print("Number columns after one-hot encoding:", len(d.columns))

Number columns after one-hot encoding: 721


'func_stat_trf', 'func_stat_trr', 'thoracic_dgn', 'prior_card_surg_type_trr', 'cod', 'func_stat_tcr', 'tcr_dgn' have many unique values.